In [ ]:
pwd # checking the directory

In [ ]:
# import libraries
import os

import cv2
import numpy as np
from keras import layers, optimizers
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model
import time

In [ ]:
def bound(low, high, value):   # bounding value between low and high
    return min(high, max(low, value))

In [ ]:
size = 100000   # size of the circle dataset for training

In [ ]:
img = np.zeros([size,256,256,3],dtype=np.uint8) # creating training samples of images of (256, 256, 3) dimensions
img.fill(255)  # for white background

# Window name in which image is displayed
window_name = 'Image'

directory = '/home/image_training/circle_dataset'   # directory where all the images will be saved

center_coordinate = np.random.rand(size,2)  # initialization of center coordinates 
center_coordinate *= 240   # converting center coordinate to (240,240) dimension for avoiding near boundary

for i in range(0,size):
    center_coordinate[i,0] = bound(30, 230, center_coordinate[i,0])
    center_coordinate[i,1] = bound(30, 230, center_coordinate[i,1])

center_coordinates = np.zeros((size,2))
top_left = np.zeros((size,2))   # initialization of top left point in the bounding boxes
bottom_right = np.zeros((size,2))  # initialization of bottom right point in the bounding boxes

radius = 20  # defining radius of the circles
center_coordinates = center_coordinate.astype(int)   # converting center coordinate into integer

top_left = center_coordinates - radius   # defining of top left point in the bounding boxes
bottom_right = center_coordinates + radius    # defining of bottom right point in the bounding boxes

color = (0, 0, 0)  # for black color
  
thickness = -1  # for filling the circle with black color

for i in range(0, size):
    image = cv2.circle(img[i], center_coordinates[i], radius, color, thickness)  # creation of training samples including black circle with white background 
    os.chdir(directory)   # changing directory to the directory where all the images will be saved
    filename = 'circle' + str(i + 1) + '.png'   # naming the image files
    cv2.imwrite(filename, image)   # writing and saving the images files in the directory

In [ ]:
np.savetxt("top_left.csv", top_left, delimiter=",")    # saving the top left points of the bounding boxes as a csv file
np.savetxt("bottom_right.csv", bottom_right, delimiter=",")  # saving the bottom right points of the bounding boxes as a csv file

In [ ]:
pwd  # checking the directory

In [ ]:
os.chdir('/home/image_training/circle_dataset')  # changing the directory where all the images were saved

In [ ]:
from csv import reader   # importing csv reader
opened_file = open('top_left.csv')   # opening the top_left csv file
read_file = reader(opened_file)  # reading the csv file
tp_lft = list(read_file)   # creation of a list with the csv file

In [ ]:
from csv import reader    # importing csv reader
opened_file1 = open('bottom_right.csv')   # opening the bottom right csv file
read_file1 = reader(opened_file1)    # reading the csv file
btm_rt = list(read_file1)   # creation of a list with the csv file

In [ ]:
folder = '/home/image_training/circle_dataset'   # directory of the saved image
circle_data = np.zeros([size,256,256,3],dtype=np.uint8)  # initializing circles array
for i in range(0,size):
    imagename = 'circle' + str(i + 1) + '.png'   # circle name
    circle_data[i,:,:,:] = cv2.imread(os.path.join(folder,imagename))  # reading the circle joining the directory and the name
    circle_data[i,:,:,:] = circle_data[i,:,:,:] / 255    # normalizing the image pixels

In [ ]:
cv2.imshow("image", circle_data[10]* 255)  # showing the circle checking whether the circle was successfully read.
while(True):
    k = cv2.waitKey(33)
    if k == -1:  # if no key was pressed, -1 is returned
        continue
    else:
        break
cv2.destroyAllWindows()

In [ ]:
bounding_box_circle = np.zeros((size,4))  # intialization of bounding box

In [ ]:
for i in range(0,size):
    bounding_box_circle[i,0] = float(tp_lft[i][0]) / 256   # coverting the top left point x coordinate to float then normalizing 
    bounding_box_circle[i,1] = float(tp_lft[i][1]) / 256   # coverting the top left point y coordinate to float then normalizing 
    bounding_box_circle[i,2] = float(btm_rt[i][0]) / 256   # coverting the bottom right point x coordinate to float then normalizing 
    bounding_box_circle[i,3] = float(btm_rt[i][1]) / 256   # coverting the bottom right point y coordinate to float then normalizing 

In [ ]:
bounding_box_circle = bounding_box_circle.astype(int)  # converting the bounding box points to integer

In [ ]:
mini = 32 # images in each batch
batch =  3125 #3125 # number of batches
epoches = 10 # number of epochs

In [ ]:
#circle detection model training 
tic = time.time()   # start time
# training process
for j in range(0, epoches):   # for loop for epochs
    for i in range(0, batch):        # for loop for batch
        start = i*mini   # starting index for each batch
        end = (i + 1)*mini  # ending index for each batch
        images_batch = circle_data[start:end,:,:,:]   # slicing image for each batch
        box_batch = bounding_box_circle[start:end,:]  # slicing  bounding box for each batch
        print("No of epochs running = " + str(j + 1) + " and No of batch running = " + str(i + 1))   # printing progress
    
        if i == 0 and j == 0:    # for 1st iteration initialization of the model
            model_c = tf.keras.models.Sequential([
                tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(256, 256, 3)),
                tf.keras.layers.MaxPooling2D(2,2),
                tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                tf.keras.layers.MaxPooling2D(2,2), 
                tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
                tf.keras.layers.MaxPooling2D(2,2),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(256, activation='relu'), 
                tf.keras.layers.Dense(128, activation='relu'),
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dense(32, activation='relu'),
                tf.keras.layers.Dense(4, activation='sigmoid') 
                ])
            model_c.summary()     # model structure
            model_c.compile(                             # setting optimizer, loss function and metrics
                            optimizer='sgd',
                            loss='mse',
                            metrics=[tf.keras.metrics.MeanSquaredError()])

            history = model_c.fit(x = images_batch,     # running the model for first batch
                                  y = box_batch,
                                  steps_per_epoch=1,
                                  epochs=1,
                                  verbose=1)
        
            model_c.save('circle_detection_model')      #  saving the model
            model_c.save_weights('circle_detection_model_weights.h5')    # saving the weights
        
        else:
            model_c = load_model('circle_detection_model')    # loading the model
            model_c.load_weights('circle_detection_model_weights.h5')    # loading the weights
            history = model_c.fit(  x = images_batch,    # running the model for rest of the images
                                  y = box_batch,
                                  steps_per_epoch=1,
                                  epochs=1,
                                  verbose=1)
            model_c.save('circle_detection_model')            # saving the model
            model_c.save_weights('circle_detection_model_weights.h5')   # saving the weights
        
toc = time.time()   # end time
print(" Time elapsed = " + str((toc - tic)/60) + "minutes")   # time elapsed

In [ ]:
model_c = load_model('circle_detection_model')    # loading the model
model_c.load_weights('circle_detection_model_weights.h5')    # loading the weights

In [ ]:
# training accuracy

begin = 1000   # starting index for evaluating training accuracy
final = 1100   # ending index for evaluating training accuracy
num = 50     # index which is used for visualization of training accuracy
preds = model_c.predict(circle_data[begin:final])  # prediction for images starting from begin to final
preds1 = preds[num] # taking specific index for visualization
print(preds1)
preds1 = preds1*256   # converting prediction bounding box from normalized to original value
box_pred = preds1.astype(int) # converting the values to integer
print(box_pred)
imgs = cv2.rectangle(circle_data[begin + num] * 255,box_pred[0:2],box_pred[2:4],(0,0,255),2) # showing the original circle along with predicted bounding box
cv2.imshow("Output", imgs)
while(True):
    k = cv2.waitKey(33)
    if k == -1:  # if no key was pressed, -1 is returned else closing the image window if any key is pressed
        continue
    else:
        break
cv2.destroyAllWindows() # closing the image window

In [ ]:
# testing accuracy

size_t = 10   # size of test samples

img_t = np.zeros([size_t,256,256,3],dtype=np.uint8)   # creating testing samples of images of (256, 256, 3) dimensions
img_t.fill(255) # for white background 

center_coordinate_t = np.random.rand(size_t,2)  # initialization of center coordinates 
center_coordinate_t *= 240 #converting center coordinate to (240,240) dimension for avoiding near boundary

for i in range(0,size_t):
    center_coordinate_t[i,0] = bound(30, 230, center_coordinate_t[i,0])  # bounding center coordinate for avoiding near boundary
    center_coordinate_t[i,1] = bound(30, 230, center_coordinate_t[i,1]) # bounding center coordinate for avoiding near boundary

center_coordinates_t = np.zeros((size_t,2))

radius_t = 20  # defining radius of the circles
center_coordinates_t = center_coordinate_t.astype(int) # converting center coordinate into integer


color_t = (0, 0, 0) # for black color
  
thickness_t = -1 # for filling the circle with black color

circle_data_t = np.zeros([size_t,256,256,3],dtype=np.uint8) # initializing circles array

for i in range(0, size_t):
    image_t = cv2.circle(img_t[i], center_coordinates_t[i], radius_t, color_t, thickness_t) # creation of test samples including black circle with white background 
    circle_data_t[i,:,:,:] = image_t / 255  # normalizing the image pixels
    
    

num_t = 0  # index of a specific image for visualization 
preds_t = model_c.predict(circle_data_t)  # prediction of test samples
preds1_t = preds_t[num_t] # predicted bounding box for the specific image
print(preds1_t)
preds1_t = preds1_t*256 # converting the predicted bounding box to its original value 
box_pred_t = preds1_t.astype(int)
print(box_pred_t)
imgs_t = cv2.rectangle(circle_data_t[num_t] * 255,box_pred_t[0:2],box_pred_t[2:4],(0,0,255),2) # showing the original circle along with predicted bounding box
cv2.imshow("Output_t", imgs_t)
while(True):
    k = cv2.waitKey(33)
    if k == -1:  # if no key was pressed, -1 is returned else closing the image window if any key is pressed
        continue
    else:
        break
cv2.destroyAllWindows()